In [19]:
import numpy as np 
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.model_selection import KFold,RepeatedStratifiedKFold,cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from google.colab import files
import io

Datasets

In [4]:
train = files.upload()

Saving train.csv to train.csv


In [5]:
valid = files.upload()

Saving valid.csv to valid.csv


In [6]:
dataset = pd.read_csv(io.StringIO(train['train.csv'].decode('utf-8')))

dataset.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [7]:
dataset.sort_values('CreationDate',inplace=True)


dataset.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [8]:
dataset.Y.value_counts().to_dict()
dataset['Y']=dataset.Y.map({'LQ_CLOSE': 0, 'LQ_EDIT': 1, 'HQ': 2})
dataset

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,0
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,2
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,2
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,2
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,2
...,...,...,...,...,...,...
44995,60461435,Convert List<String> to string C# - asp.net - ...,<p>I am new to this and I am asking for help t...,<c#><asp.net><sql-server>,2020-02-29 02:22:18,0
44996,60461754,Does Python execute code from the top or botto...,<p>I am working on learning Python and was won...,<python>,2020-02-29 03:33:59,0
44997,60462001,how to change payment date in Azure?,<p>It looks like it costs 8 days per month in ...,<azure><billing>,2020-02-29 04:34:16,0
44998,60465318,how to implement fill in the blank in Swift,"<p>""I _____ any questions.""</p>\n\n<p>I want t...",<ios><swift>,2020-02-29 12:50:43,0


In [9]:
import re
def clean_tags(T):
    T=T.lower()
    text=re.sub(r'<','',T)
    text=re.sub(r'>',' ',text)
    return text

dataset['Tags']=dataset['Tags'].map(clean_tags)
dataset.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,java repeat,2016-01-01 00:21:59,0
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,java optional,2016-01-01 02:03:20,2
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,javascript image overlay react-native opacity,2016-01-01 02:48:24,2
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",swift operators whitespace ternary-operator op...,2016-01-01 03:30:17,2
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,android material-design floating-action-button,2016-01-01 05:21:48,2


In [ ]:
dataset.Tags.value_counts()[:10]

python                798
java                  761
javascript            660
c#                    531
c++                   521
c                     435
php                   423
android               392
html css              306
javascript jquery     273
Name: Tags, dtype: int64

Об'єднання тексту і назви і деякі модифікації


In [25]:
import os
def clean_text(x):
    x=x.lower()
    x=re.sub(r'[^(a-zA-Z)\s]','', x)
    return x

In [30]:
dataset['TitleBodyTags'] = dataset['Title'] + ' ' + dataset['Body'] + ' ' + dataset['Tags']
dataset['TitleBodyTags']= dataset.TitleBodyTags.map(clean_text)
dataset.head()

,Id,Title,Body,Tags,CreationDate,TitleBodyTags
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,java repeat,2016-01-01 00:21:59,java repeat task every random seconds pim alre...
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,java optional,2016-01-01 02:03:20,why are java optionals immutable pid like to u...
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,javascript image overlay react-native opacity,2016-01-01 02:48:24,text overlay image with darkened opacity react...
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",swift operators whitespace ternary-operator op...,2016-01-01 03:30:17,why ternary operator in swift is so picky pthe...
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,android material-design floating-action-button,2016-01-01 05:21:48,hideshow fab with scale animation pim using cu...


In [12]:
label=dataset.pop('Y')
dataset.head()

,Id,Title,Body,Tags,CreationDate,CombineTextandBody,TitleBodyTags
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,java repeat,2016-01-01 00:21:59,java repeat task every random seconds pim alre...,java repeat task every random seconds pim alre...
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,java optional,2016-01-01 02:03:20,why are java optionals immutable pid like to u...,why are java optionals immutable pid like to u...
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,javascript image overlay react-native opacity,2016-01-01 02:48:24,text overlay image with darkened opacity react...,text overlay image with darkened opacity react...
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",swift operators whitespace ternary-operator op...,2016-01-01 03:30:17,why ternary operator in swift is so picky pthe...,why ternary operator in swift is so picky pthe...
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,android material-design floating-action-button,2016-01-01 05:21:48,hideshow fab with scale animation pim using cu...,hideshow fab with scale animation pim using cu...


Validation dataset

In [32]:
validDataset = pd.read_csv(io.StringIO(valid['valid.csv'].decode('utf-8')))
validDataset.sort_values('CreationDate',inplace=True)
validDataset.Y.value_counts().to_dict()
validDataset['Y']=validDataset.Y.map({'LQ_CLOSE': 0, 'LQ_EDIT': 1, 'HQ': 2})

In [34]:

validDataset['TitleBodyTags'] = validDataset['Title'] + ' ' + validDataset['Body'] + ' ' + validDataset['Tags']
validDataset['TitleBodyTags']= validDataset.TitleBodyTags.map(clean_text)
validDataset.head()

,Id,Title,Body,Tags,CreationDate,Y,TitleBodyTags
0,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,1,how to get all the child records from differen...
1,34554721,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,<php><mysql><sql><codeigniter><mysqli>,2016-01-01 08:43:50,1,retrieve all except some data of the another t...
2,34555135,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,<python><pandas>,2016-01-01 09:55:22,2,pandas readhtml pim trying to extract us state...
3,34555448,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",<sql-server><c#-4.0>,2016-01-01 10:43:45,1,reader always gimme null im so new to c i wann...
4,34555752,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,<php>,2016-01-01 11:34:09,1,php rearrange array elements based on conditio...


In [35]:
train_x,test_x,train_y,test_y=train_test_split(dataset,label,test_size=0.15,random_state=42)

In [47]:
train_x.shape,test_x.shape

((38250, 6), (6750, 6))

In [37]:
tfidf=TfidfVectorizer()
transform_text_train=tfidf.fit_transform(train_x.TitleBodyTags)

transform_text_test=tfidf.transform(test_x.TitleBodyTags)

Validation Dataset

In [39]:
transform_text_validation = tfidf.transform(validDataset.TitleBodyTags)

*Logistic* Regression

In [52]:
lr_classifier = LogisticRegression()

lr_classifier.fit(transform_text_train, train_y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Test validation


In [53]:
score=cross_val_score(lr_classifier,transform_text_train, train_y,cv=3,n_jobs=-1)
score

array([0.87270588, 0.86988235, 0.86713725])

Validation by valid train

In [54]:
scoreValidation=cross_val_score(lr_classifier, transform_text_validation, validDataset.Y, cv=3,n_jobs=-1)
scoreValidation

array([0.8264, 0.8474, 0.8364])